In [45]:
import MetaTrader5 as mt5
import pandas as pd
import numpy as np
from datetime import datetime
import time
import os

# Manual RSI and ATR calculation functions to replace pandas_ta
def calculate_rsi(prices, period=14):
    """Calculate RSI manually"""
    delta = prices.diff()
    gain = delta.where(delta > 0, 0)
    loss = -delta.where(delta < 0, 0)
    
    avg_gain = gain.rolling(window=period).mean()
    avg_loss = loss.rolling(window=period).mean()
    
    rs = avg_gain / avg_loss
    rsi = 100 - (100 / (1 + rs))
    return rsi

def calculate_atr(high, low, close, period=14):
    """Calculate ATR manually"""
    tr1 = high - low
    tr2 = abs(high - close.shift())
    tr3 = abs(low - close.shift())
    tr = pd.DataFrame({'tr1': tr1, 'tr2': tr2, 'tr3': tr3}).max(axis=1)
    atr = tr.rolling(window=period).mean()
    return atr

# --- MT5 Connection ---
if not mt5.initialize():
    print("initialize() failed, error code =", mt5.last_error())
    quit()

print("MetaTrader5 package version:", mt5.version())
# Display account info
account_info = mt5.account_info()
if account_info:
    print("Login:", account_info.login)
    print("Server:", account_info.server)
    print("Balance:", account_info.balance, account_info.currency)
else:
    print("Failed to get account info, error code =", mt5.last_error())


MetaTrader5 package version: (500, 5120, '13 Jun 2025')
Login: 544482057
Server: Dukascopy-demo-mt5-1
Balance: 10000.0 USD


In [46]:
# --- Strategy Parameters ---
TIMEFRAME = mt5.TIMEFRAME_M5
START_DATE = datetime(2020, 1, 1)
END_DATE = datetime.now()
RSI_PERIOD = 14
ATR_PERIOD = 5
RSI_OVERBOUGHT = 75
RSI_OVERSOLD = 25

# --- USD-Based Risk Management Parameters ---
PROFIT_TARGET_USD = 500.0      # Target profit in USD per trade
STOP_LOSS_USD = -15000.0         # Maximum loss in USD per trade  
MAX_TRADE_HOURS = 2400           # Maximum trade duration in hours
BASE_LOT_SIZE = 1.0            # Base lot size for symbol1

# --- List of Pairs to Test ---
# Format: (SYMBOL_1, SYMBOL_2, correlation_coefficient)
PAIRS_TO_TEST = [
    # Test pairs with different instrument types for validation
    ('USDJPY', 'AUDUSD', -0.2529543842268859),  # JPY vs USD pair
    ('EURUSD', 'GBPUSD', -0.3),  # Major vs Major
    ('XAUUSD', 'XAGUSD', -0.25),  # Gold vs Major (if available)
    ('USDJPY', 'NZDCAD', -0.2501672888948474),
    ('NZDCAD', 'CADJPY', -0.25133057567406397),
    ('USDCHF', 'AUDCAD', -0.2562165879569482),
    ('USDJPY', 'EURCAD', -0.25847348145620475),
    ('USDCHF', 'GBPCAD', -0.271396417881951),
    ('EURGBP', 'GBPCAD', -0.27171511851852664),
    ('USDCAD', 'CADCHF', -0.2777462293886388),
    ('AUDNZD', 'NZDCAD', -0.2822730661465801),
    ('EURAUD', 'CADCHF', -0.28774716861182736),
    ('USDCAD', 'CADJPY', -0.2879791860947295),
    ('EURAUD', 'NZDCAD', -0.2889060336087516),
    ('EURGBP', 'GBPJPY', -0.2894033148372946),
    ('GBPAUD', 'NZDCAD', -0.28960693776721835),
    ('USDJPY', 'NZDUSD', -0.30922549779820757),
    ('GBPAUD', 'AUDNZD', -0.3107427713136811),
    ('USDCHF', 'NZDCAD', -0.31783843161158903),
    ('EURNZD', 'AUDJPY', -0.31905157076312995),
    ('EURAUD', 'AUDNZD', -0.3298622040571091),
    ('GBPUSD', 'USDJPY', -0.33103707942732463),
    ('GBPNZD', 'AUDCHF', -0.3325297119495989),
    ('USDCAD', 'NZDCHF', -0.33626329143224054),
    ('USDCAD', 'NZDJPY', -0.34014011220752294),
    ('GBPNZD', 'AUDCAD', -0.34295512381677884),
    ('GBPCAD', 'CADJPY', -0.34388652196712044),
    ('USDCAD', 'AUDJPY', -0.35157909544893895),
    ('GBPAUD', 'NZDJPY', -0.352627087360886),
    ('USDCAD', 'AUDCHF', -0.3590176972472969),
    ('EURNZD', 'AUDCAD', -0.37229618712286894),
    ('EURUSD', 'USDJPY', -0.37466311988879697),
    ('GBPNZD', 'NZDJPY', -0.3800492152373967),
    ('EURAUD', 'NZDJPY', -0.38796792975442396),
    ('USDCHF', 'EURCAD', -0.3971711068832913),
    ('EURCAD', 'CADJPY', -0.40139681921779524),
    ('GBPAUD', 'NZDCHF', -0.41230327324745386),
    ('NZDUSD', 'EURAUD', -0.4128613735426008),
    ('GBPCAD', 'CADCHF', -0.41450090984024557),
    ('AUDUSD', 'GBPNZD', -0.42390417722083973),
    ('NZDUSD', 'GBPAUD', -0.42812428197260166),
    ('USDCHF', 'AUDUSD', -0.4296189558929426),
    ('AUDUSD', 'EURNZD', -0.4475936607587085),
    ('EURNZD', 'NZDJPY', -0.44965878256485514),
    ('EURGBP', 'GBPCHF', -0.4562188512282561),
    ('EURNZD', 'AUDCHF', -0.4697224793660116),
    ('GBPNZD', 'NZDCHF', -0.4764004659426721),
    ('GBPAUD', 'AUDJPY', -0.47825832508878097),
    ('USDCHF', 'NZDUSD', -0.4839228548195487),
    ('GBPNZD', 'NZDCAD', -0.48657646905729446),
    ('EURNZD', 'NZDCAD', -0.5022229576817127),
    ('EURAUD', 'NZDCHF', -0.5132508230887801),
    ('EURAUD', 'AUDJPY', -0.5195399029165325),
    ('GBPAUD', 'AUDCAD', -0.5246859027090179),
    ('EURAUD', 'AUDCAD', -0.5271486785019794),
    ('GBPUSD', 'USDCHF', -0.5453332767389073),
    ('EURCAD', 'CADCHF', -0.5486137520730775),
    ('NZDUSD', 'GBPNZD', -0.5531454087942047),
    ('EURUSD', 'USDCAD', -0.5554541896798381),
    ('GBPUSD', 'USDCAD', -0.5639746107281004),
    ('NZDUSD', 'EURNZD', -0.5675948585931192),
    ('GBPAUD', 'AUDCHF', -0.5742895035218368),
    ('AUDUSD', 'EURAUD', -0.5932228622719815),
    ('AUDUSD', 'GBPAUD', -0.6024946865592142),
    ('EURNZD', 'NZDCHF', -0.6160512976019872),
    ('USDCAD', 'NZDUSD', -0.6352592735085854),
    ('EURUSD', 'USDCHF', -0.6644410206546534),
    ('AUDUSD', 'USDCAD', -0.6703918566349919),
    ('EURAUD', 'AUDCHF', -0.6754847892250637)
]

print(f"Loaded {len(PAIRS_TO_TEST)} pairs for backtesting.")


Loaded 69 pairs for backtesting.


In [47]:
def run_backtest(symbol1, symbol2):
    """
    Main function to run the backtest for a given pair of symbols.
    Now uses USD-based profit targets with comprehensive risk management.
    """
    print(f"\n----- Starting Backtest for {symbol1} / {symbol2} -----")
    
    # Get account balance for risk-based position sizing
    account_balance = get_account_balance()
    print(f"Account Balance: ${account_balance:,.2f}")
    
    # 1. Fetch Data
    df1 = get_historical_data(symbol1, TIMEFRAME, START_DATE, END_DATE)
    df2 = get_historical_data(symbol2, TIMEFRAME, START_DATE, END_DATE)

    if df1.empty or df2.empty:
        print(f"Could not fetch data for one of the symbols. Skipping pair.")
        return

    # 2. Combine data and calculate indicators
    df = pd.DataFrame(index=df1.index.union(df2.index))
    df['s1_close'] = df1['close']
    df['s2_close'] = df2['close']
    df.dropna(inplace=True) # Ensure we only have timestamps where both pairs have data

    df['s1_rsi'] = calculate_rsi(df['s1_close'], RSI_PERIOD)
    df['s2_rsi'] = calculate_rsi(df['s2_close'], RSI_PERIOD)
    df['s1_atr'] = calculate_atr(df1['high'], df1['low'], df1['close'], ATR_PERIOD)
    df['s2_atr'] = calculate_atr(df2['high'], df2['low'], df2['close'], ATR_PERIOD)
    df.dropna(inplace=True)

    print(f"Data prepared. Starting simulation with {len(df)} bars.")

    # 3. Simulation Loop
    trade_history = []
    trade_id_counter = 1
    in_trade = False
    current_trade = {}

    for i in range(len(df)):
        # --- CHECK EXIT CONDITIONS ---
        if in_trade:
            # Use the new USD-based exit condition check
            exit_result = check_exit_conditions(
                current_trade, 
                symbol1, 
                symbol2, 
                df.index[i], 
                df['s1_close'].iloc[i], 
                df['s2_close'].iloc[i]
            )
            
            if exit_result is not None:
                exit_reason, total_pnl, s1_pnl, s2_pnl = exit_result
                
                # Close the trade
                current_trade['exit_time'] = df.index[i]
                current_trade['s1_exit_price'] = df['s1_close'].iloc[i]
                current_trade['s2_exit_price'] = df['s2_close'].iloc[i]
                current_trade['exit_reason'] = exit_reason

                duration = (current_trade['exit_time'] - current_trade['entry_time']).total_seconds() / 3600

                # Calculate additional metrics for enhanced reporting
                s1_pips = calculate_pips(symbol1, current_trade['s1_entry_price'], current_trade['s1_exit_price'], current_trade['type'])
                s2_pips = calculate_pips(symbol2, current_trade['s2_entry_price'], current_trade['s2_exit_price'], current_trade['type'])
                
                trade_history.append({
                    # Basic Trade Info
                    "ID": current_trade['id'],
                    "Trade Type": current_trade['type'].upper(),
                    "Entry Time": current_trade['entry_time'],
                    "Exit Time": current_trade['exit_time'],
                    "Duration (hrs)": round(duration, 2),
                    "Exit Reason": exit_reason,
                    
                    # Symbol Information
                    "Symbol1": symbol1,
                    "Symbol2": symbol2,
                    
                    # Entry Conditions (stored during trade opening)
                    "Symbol1 Entry RSI": current_trade.get('s1_entry_rsi', 'N/A'),
                    "Symbol2 Entry RSI": current_trade.get('s2_entry_rsi', 'N/A'),
                    "Symbol1 Entry ATR": current_trade.get('s1_entry_atr', 'N/A'),
                    "Symbol2 Entry ATR": current_trade.get('s2_entry_atr', 'N/A'),
                    
                    # Price Information  
                    "Symbol1 Entry": current_trade['s1_entry_price'],
                    "Symbol1 Exit": current_trade['s1_exit_price'],
                    "Symbol2 Entry": current_trade['s2_entry_price'],
                    "Symbol2 Exit": current_trade['s2_exit_price'],
                    
                    # Position Sizing
                    "Symbol1 Lots": current_trade['s1_lots'],
                    "Symbol2 Lots": current_trade['s2_lots'],
                    "Hedge Ratio": current_trade.get('hedge_ratio', 'N/A'),
                    "Risk Amount USD": current_trade.get('risk_amount', 'N/A'),
                    "Account Balance": current_trade.get('account_balance', 'N/A'),
                    
                    # Performance Metrics
                    "Symbol1 Pips": round(s1_pips, 1),
                    "Symbol2 Pips": round(s2_pips, 1),
                    "Total Pips": round(s1_pips + s2_pips, 1),
                    "Symbol1 P&L": round(s1_pnl, 2),
                    "Symbol2 P&L": round(s2_pnl, 2),
                    "Total P&L": round(total_pnl, 2),
                    "Return %": round((total_pnl / current_trade.get('risk_amount', 100)) * 100, 2) if current_trade.get('risk_amount', 0) > 0 else 'N/A'
                })
                
                print(f"  Trade {current_trade['id']} closed: {exit_reason} | Total P&L: ${total_pnl:.2f} | Duration: {duration:.1f}h")
                
                in_trade = False
                current_trade = {}
                continue # Move to next bar after closing

        # --- CHECK ENTRY CONDITIONS ---
        if not in_trade:
            trade_type_to_open = None
            
            # Signal A: Short both pairs
            if df['s1_rsi'].iloc[i] > RSI_OVERBOUGHT and df['s2_rsi'].iloc[i] > RSI_OVERBOUGHT:
                trade_type_to_open = 'short'
            
            # Signal B: Long both pairs
            elif df['s1_rsi'].iloc[i] < RSI_OVERSOLD and df['s2_rsi'].iloc[i] < RSI_OVERSOLD:
                trade_type_to_open = 'long'

            if trade_type_to_open:
                # Calculate ATR values
                s1_atr_val = df['s1_atr'].iloc[i]
                s2_atr_val = df['s2_atr'].iloc[i]
                
                if s1_atr_val > 0 and s2_atr_val > 0:
                    # NEW: Use risk-based position sizing
                    s1_lot_size, s2_lot_size = calculate_risk_based_lots(
                        symbol1, symbol2, account_balance, s1_atr_val, s2_atr_val
                    )
                    
                    # Calculate additional metrics for storage
                    s1_pip_size = get_pip_size(symbol1)
                    s2_pip_size = get_pip_size(symbol2)
                    s1_atr_pips = s1_atr_val / s1_pip_size
                    s2_atr_pips = s2_atr_val / s2_pip_size
                    
                    # Calculate risk and hedge ratio
                    risk_usd = account_balance * (RISK_PER_TRADE_PCT / 100)
                    hedge_ratio = calculate_hedge_ratio(symbol1, symbol2, s1_atr_val, s2_atr_val)
                    
                    print(f"Entry Signal: {trade_type_to_open}")
                    print(f"  Risk per trade: ${risk_usd:.2f} ({RISK_PER_TRADE_PCT}% of balance)")
                    print(f"  {symbol1}: RSI={df['s1_rsi'].iloc[i]:.1f} | ATR={s1_atr_val:.5f} ({s1_atr_pips:.1f} pips) | Lot size: {s1_lot_size}")
                    print(f"  {symbol2}: RSI={df['s2_rsi'].iloc[i]:.1f} | ATR={s2_atr_val:.5f} ({s2_atr_pips:.1f} pips) | Lot size: {s2_lot_size}")
                    print(f"  Target: ${PROFIT_TARGET_USD} | Stop: ${STOP_LOSS_USD} | Max time: {MAX_TRADE_HOURS}h")
                    
                    # Open the trade with comprehensive information storage
                    in_trade = True
                    current_trade = {
                        # Basic trade info
                        'id': trade_id_counter,
                        'type': trade_type_to_open,
                        'entry_time': df.index[i],
                        
                        # Price and position info
                        's1_entry_price': df['s1_close'].iloc[i],
                        's2_entry_price': df['s2_close'].iloc[i],
                        's1_lots': s1_lot_size,
                        's2_lots': s2_lot_size,
                        
                        # Entry conditions for analysis
                        's1_entry_rsi': round(df['s1_rsi'].iloc[i], 1),
                        's2_entry_rsi': round(df['s2_rsi'].iloc[i], 1),
                        's1_entry_atr': round(s1_atr_val, 5),
                        's2_entry_atr': round(s2_atr_val, 5),
                        
                        # Risk management info
                        'hedge_ratio': round(hedge_ratio, 4),
                        'risk_amount': round(risk_usd, 2),
                        'account_balance': round(account_balance, 2)
                    }
                    trade_id_counter += 1

    # 4. Save Results
    if not trade_history:
        print("No trades were executed for this pair.")
    else:
        report_df = pd.DataFrame(trade_history)
        output_filename = f"{symbol1}_{symbol2}_backtest_report.csv"
        report_df.to_csv(output_filename, index=False)
        print(f"Backtest complete. Report saved to: {output_filename}")
        print("--- Summary ---")
        print(f"Total Trades: {len(report_df)}")
        print(f"Total P&L: ${report_df['Total P&L'].sum():.2f}")
        
        # Enhanced statistics and analysis
        profit_trades = report_df[report_df['Total P&L'] > 0]
        loss_trades = report_df[report_df['Total P&L'] < 0]
        
        print(f"\n--- Detailed Performance Analysis ---")
        print(f"Winning Trades: {len(profit_trades)} ({len(profit_trades)/len(report_df)*100:.1f}%)")
        print(f"Losing Trades: {len(loss_trades)} ({len(loss_trades)/len(report_df)*100:.1f}%)")
        print(f"Average Duration: {report_df['Duration (hrs)'].mean():.1f} hours")
        
        # P&L Statistics
        if len(profit_trades) > 0:
            print(f"Average Win: ${profit_trades['Total P&L'].mean():.2f}")
            print(f"Largest Win: ${profit_trades['Total P&L'].max():.2f}")
        if len(loss_trades) > 0:
            print(f"Average Loss: ${loss_trades['Total P&L'].mean():.2f}")
            print(f"Largest Loss: ${loss_trades['Total P&L'].min():.2f}")
        
        # Exit reason breakdown
        exit_reasons = report_df['Exit Reason'].value_counts()
        print(f"\nExit Reasons: {dict(exit_reasons)}")
        
        # Risk-Return Analysis
        try:
            returns_data = report_df[report_df['Return %'] != 'N/A']['Return %']
            if len(returns_data) > 0:
                numeric_returns = [float(x) for x in returns_data if str(x).replace('.', '').replace('-', '').isdigit()]
                if numeric_returns:
                    print(f"\nRisk-Return Analysis:")
                    print(f"Average Return per Trade: {sum(numeric_returns)/len(numeric_returns):.1f}%")
                    print(f"Best Return: {max(numeric_returns):.1f}%")
                    print(f"Worst Return: {min(numeric_returns):.1f}%")
        except (ValueError, TypeError):
            print(f"\nRisk-Return Analysis: Data not available")
        
        # Entry Condition Analysis
        long_trades = report_df[report_df['Trade Type'] == 'LONG']
        short_trades = report_df[report_df['Trade Type'] == 'SHORT']
        
        print(f"\nEntry Signal Analysis:")
        print(f"Long Trades: {len(long_trades)} | Short Trades: {len(short_trades)}")
        
        if len(long_trades) > 0:
            try:
                s1_rsi_values = [float(x) for x in long_trades['Symbol1 Entry RSI'] if str(x) != 'N/A']
                s2_rsi_values = [float(x) for x in long_trades['Symbol2 Entry RSI'] if str(x) != 'N/A']
                if s1_rsi_values and s2_rsi_values:
                    avg_rsi1_long = sum(s1_rsi_values) / len(s1_rsi_values)
                    avg_rsi2_long = sum(s2_rsi_values) / len(s2_rsi_values)
                    print(f"Long Trades - Avg Entry RSI: {symbol1}={avg_rsi1_long:.1f}, {symbol2}={avg_rsi2_long:.1f}")
            except (ValueError, TypeError):
                print(f"Long Trades - RSI data not available")
            
        if len(short_trades) > 0:
            try:
                s1_rsi_values = [float(x) for x in short_trades['Symbol1 Entry RSI'] if str(x) != 'N/A']
                s2_rsi_values = [float(x) for x in short_trades['Symbol2 Entry RSI'] if str(x) != 'N/A']
                if s1_rsi_values and s2_rsi_values:
                    avg_rsi1_short = sum(s1_rsi_values) / len(s1_rsi_values)
                    avg_rsi2_short = sum(s2_rsi_values) / len(s2_rsi_values)
                    print(f"Short Trades - Avg Entry RSI: {symbol1}={avg_rsi1_short:.1f}, {symbol2}={avg_rsi2_short:.1f}")
            except (ValueError, TypeError):
                print(f"Short Trades - RSI data not available")
        
        # Position Sizing Analysis
        try:
            avg_s1_lots = sum(report_df['Symbol1 Lots']) / len(report_df['Symbol1 Lots'])
            avg_s2_lots = sum(report_df['Symbol2 Lots']) / len(report_df['Symbol2 Lots'])
            hedge_ratios = [float(x) for x in report_df['Hedge Ratio'] if str(x) != 'N/A']
            avg_hedge_ratio = sum(hedge_ratios) / len(hedge_ratios) if hedge_ratios else 0
            
            print(f"\nPosition Sizing Analysis:")
            print(f"Average Lot Sizes: {symbol1}={avg_s1_lots:.2f}, {symbol2}={avg_s2_lots:.2f}")
            print(f"Average Hedge Ratio: {avg_hedge_ratio:.3f}")
        except (ValueError, TypeError, ZeroDivisionError):
            print(f"\nPosition Sizing Analysis: Data not available")
        
        # Pips vs USD Analysis
        total_pips = report_df['Total Pips'].sum()
        total_usd = report_df['Total P&L'].sum()
        print(f"\nPips vs USD Comparison:")
        print(f"Total Pips: {total_pips:.1f} | Total USD: ${total_usd:.2f}")
        print(f"Average USD per Pip: ${total_usd/total_pips:.2f}" if total_pips != 0 else "No pip data available")


In [48]:
def run_backtest(symbol1, symbol2):
    """
    Main function to run the backtest for a given pair of symbols.
    Now uses USD-based profit targets with comprehensive risk management.
    """
    print(f"\n----- Starting Backtest for {symbol1} / {symbol2} -----")
    
    # Get account balance for risk-based position sizing
    account_balance = get_account_balance()
    print(f"Account Balance: ${account_balance:,.2f}")
    
    # 1. Fetch Data
    df1 = get_historical_data(symbol1, TIMEFRAME, START_DATE, END_DATE)
    df2 = get_historical_data(symbol2, TIMEFRAME, START_DATE, END_DATE)

    if df1.empty or df2.empty:
        print(f"Could not fetch data for one of the symbols. Skipping pair.")
        return

    # 2. Combine data and calculate indicators
    df = pd.DataFrame(index=df1.index.union(df2.index))
    df['s1_close'] = df1['close']
    df['s2_close'] = df2['close']
    df.dropna(inplace=True) # Ensure we only have timestamps where both pairs have data

    df['s1_rsi'] = calculate_rsi(df['s1_close'], RSI_PERIOD)
    df['s2_rsi'] = calculate_rsi(df['s2_close'], RSI_PERIOD)
    df['s1_atr'] = calculate_atr(df1['high'], df1['low'], df1['close'], ATR_PERIOD)
    df['s2_atr'] = calculate_atr(df2['high'], df2['low'], df2['close'], ATR_PERIOD)
    df.dropna(inplace=True)

    print(f"Data prepared. Starting simulation with {len(df)} bars.")

    # 3. Simulation Loop
    trade_history = []
    trade_id_counter = 1
    in_trade = False
    current_trade = {}

    for i in range(len(df)):
        # --- CHECK EXIT CONDITIONS ---
        if in_trade:
            # Use the new USD-based exit condition check
            exit_result = check_exit_conditions(
                current_trade, 
                symbol1, 
                symbol2, 
                df.index[i], 
                df['s1_close'].iloc[i], 
                df['s2_close'].iloc[i]
            )
            
            if exit_result is not None:
                exit_reason, total_pnl, s1_pnl, s2_pnl = exit_result
                
                # Close the trade
                current_trade['exit_time'] = df.index[i]
                current_trade['s1_exit_price'] = df['s1_close'].iloc[i]
                current_trade['s2_exit_price'] = df['s2_close'].iloc[i]
                current_trade['exit_reason'] = exit_reason

                duration = (current_trade['exit_time'] - current_trade['entry_time']).total_seconds() / 3600

                # Calculate additional metrics for enhanced reporting
                s1_pips = calculate_pips(symbol1, current_trade['s1_entry_price'], current_trade['s1_exit_price'], current_trade['type'])
                s2_pips = calculate_pips(symbol2, current_trade['s2_entry_price'], current_trade['s2_exit_price'], current_trade['type'])
                
                trade_history.append({
                    # Basic Trade Info
                    "ID": current_trade['id'],
                    "Trade Type": current_trade['type'].upper(),
                    "Entry Time": current_trade['entry_time'],
                    "Exit Time": current_trade['exit_time'],
                    "Duration (hrs)": round(duration, 2),
                    "Exit Reason": exit_reason,
                    
                    # Symbol Information
                    "Symbol1": symbol1,
                    "Symbol2": symbol2,
                    
                    # Entry Conditions (stored during trade opening)
                    "Symbol1 Entry RSI": current_trade.get('s1_entry_rsi', 'N/A'),
                    "Symbol2 Entry RSI": current_trade.get('s2_entry_rsi', 'N/A'),
                    "Symbol1 Entry ATR": current_trade.get('s1_entry_atr', 'N/A'),
                    "Symbol2 Entry ATR": current_trade.get('s2_entry_atr', 'N/A'),
                    
                    # Price Information  
                    "Symbol1 Entry": current_trade['s1_entry_price'],
                    "Symbol1 Exit": current_trade['s1_exit_price'],
                    "Symbol2 Entry": current_trade['s2_entry_price'],
                    "Symbol2 Exit": current_trade['s2_exit_price'],
                    
                    # Position Sizing
                    "Symbol1 Lots": current_trade['s1_lots'],
                    "Symbol2 Lots": current_trade['s2_lots'],
                    "Hedge Ratio": current_trade.get('hedge_ratio', 'N/A'),
                    "Risk Amount USD": current_trade.get('risk_amount', 'N/A'),
                    "Account Balance": current_trade.get('account_balance', 'N/A'),
                    
                    # Performance Metrics
                    "Symbol1 Pips": round(s1_pips, 1),
                    "Symbol2 Pips": round(s2_pips, 1),
                    "Total Pips": round(s1_pips + s2_pips, 1),
                    "Symbol1 P&L": round(s1_pnl, 2),
                    "Symbol2 P&L": round(s2_pnl, 2),
                    "Total P&L": round(total_pnl, 2),
                    "Return %": round((total_pnl / current_trade.get('risk_amount', 100)) * 100, 2) if current_trade.get('risk_amount', 0) > 0 else 'N/A'
                })
                
                print(f"  Trade {current_trade['id']} closed: {exit_reason} | Total P&L: ${total_pnl:.2f} | Duration: {duration:.1f}h")
                
                in_trade = False
                current_trade = {}
                continue # Move to next bar after closing

        # --- CHECK ENTRY CONDITIONS ---
        if not in_trade:
            trade_type_to_open = None
            
            # Signal A: Short both pairs
            if df['s1_rsi'].iloc[i] > RSI_OVERBOUGHT and df['s2_rsi'].iloc[i] > RSI_OVERBOUGHT:
                trade_type_to_open = 'short'
            
            # Signal B: Long both pairs
            elif df['s1_rsi'].iloc[i] < RSI_OVERSOLD and df['s2_rsi'].iloc[i] < RSI_OVERSOLD:
                trade_type_to_open = 'long'

            if trade_type_to_open:
                # Calculate ATR values
                s1_atr_val = df['s1_atr'].iloc[i]
                s2_atr_val = df['s2_atr'].iloc[i]
                
                if s1_atr_val > 0 and s2_atr_val > 0:
                    # NEW: Use risk-based position sizing
                    s1_lot_size, s2_lot_size = calculate_risk_based_lots(
                        symbol1, symbol2, account_balance, s1_atr_val, s2_atr_val
                    )
                    
                    # Calculate additional metrics for storage
                    s1_pip_size = get_pip_size(symbol1)
                    s2_pip_size = get_pip_size(symbol2)
                    s1_atr_pips = s1_atr_val / s1_pip_size
                    s2_atr_pips = s2_atr_val / s2_pip_size
                    
                    # Calculate risk and hedge ratio
                    risk_usd = account_balance * (RISK_PER_TRADE_PCT / 100)
                    hedge_ratio = calculate_hedge_ratio(symbol1, symbol2, s1_atr_val, s2_atr_val)
                    
                    print(f"Entry Signal: {trade_type_to_open}")
                    print(f"  Risk per trade: ${risk_usd:.2f} ({RISK_PER_TRADE_PCT}% of balance)")
                    print(f"  {symbol1}: RSI={df['s1_rsi'].iloc[i]:.1f} | ATR={s1_atr_val:.5f} ({s1_atr_pips:.1f} pips) | Lot size: {s1_lot_size}")
                    print(f"  {symbol2}: RSI={df['s2_rsi'].iloc[i]:.1f} | ATR={s2_atr_val:.5f} ({s2_atr_pips:.1f} pips) | Lot size: {s2_lot_size}")
                    print(f"  Target: ${PROFIT_TARGET_USD} | Stop: ${STOP_LOSS_USD} | Max time: {MAX_TRADE_HOURS}h")
                    
                    # Open the trade with comprehensive information storage
                    in_trade = True
                    current_trade = {
                        # Basic trade info
                        'id': trade_id_counter,
                        'type': trade_type_to_open,
                        'entry_time': df.index[i],
                        
                        # Price and position info
                        's1_entry_price': df['s1_close'].iloc[i],
                        's2_entry_price': df['s2_close'].iloc[i],
                        's1_lots': s1_lot_size,
                        's2_lots': s2_lot_size,
                        
                        # Entry conditions for analysis
                        's1_entry_rsi': round(df['s1_rsi'].iloc[i], 1),
                        's2_entry_rsi': round(df['s2_rsi'].iloc[i], 1),
                        's1_entry_atr': round(s1_atr_val, 5),
                        's2_entry_atr': round(s2_atr_val, 5),
                        
                        # Risk management info
                        'hedge_ratio': round(hedge_ratio, 4),
                        'risk_amount': round(risk_usd, 2),
                        'account_balance': round(account_balance, 2)
                    }
                    trade_id_counter += 1

    # 4. Save Results
    if not trade_history:
        print("No trades were executed for this pair.")
    else:
        report_df = pd.DataFrame(trade_history)
        output_filename = f"{symbol1}_{symbol2}_backtest_report.csv"
        report_df.to_csv(output_filename, index=False)
        print(f"Backtest complete. Report saved to: {output_filename}")
        print("--- Summary ---")
        print(f"Total Trades: {len(report_df)}")
        print(f"Total P&L: ${report_df['Total P&L'].sum():.2f}")
        
        # Enhanced statistics and analysis
        profit_trades = report_df[report_df['Total P&L'] > 0]
        loss_trades = report_df[report_df['Total P&L'] < 0]
        
        print(f"\n--- Detailed Performance Analysis ---")
        print(f"Winning Trades: {len(profit_trades)} ({len(profit_trades)/len(report_df)*100:.1f}%)")
        print(f"Losing Trades: {len(loss_trades)} ({len(loss_trades)/len(report_df)*100:.1f}%)")
        print(f"Average Duration: {report_df['Duration (hrs)'].mean():.1f} hours")
        
        # P&L Statistics
        if len(profit_trades) > 0:
            print(f"Average Win: ${profit_trades['Total P&L'].mean():.2f}")
            print(f"Largest Win: ${profit_trades['Total P&L'].max():.2f}")
        if len(loss_trades) > 0:
            print(f"Average Loss: ${loss_trades['Total P&L'].mean():.2f}")
            print(f"Largest Loss: ${loss_trades['Total P&L'].min():.2f}")
        
        # Exit reason breakdown
        exit_reasons = report_df['Exit Reason'].value_counts()
        print(f"\nExit Reasons: {dict(exit_reasons)}")
        
        # Risk-Return Analysis
        try:
            returns_data = report_df[report_df['Return %'] != 'N/A']['Return %']
            if len(returns_data) > 0:
                numeric_returns = [float(x) for x in returns_data if str(x).replace('.', '').replace('-', '').isdigit()]
                if numeric_returns:
                    print(f"\nRisk-Return Analysis:")
                    print(f"Average Return per Trade: {sum(numeric_returns)/len(numeric_returns):.1f}%")
                    print(f"Best Return: {max(numeric_returns):.1f}%")
                    print(f"Worst Return: {min(numeric_returns):.1f}%")
        except (ValueError, TypeError):
            print(f"\nRisk-Return Analysis: Data not available")
        
        # Entry Condition Analysis
        long_trades = report_df[report_df['Trade Type'] == 'LONG']
        short_trades = report_df[report_df['Trade Type'] == 'SHORT']
        
        print(f"\nEntry Signal Analysis:")
        print(f"Long Trades: {len(long_trades)} | Short Trades: {len(short_trades)}")
        
        if len(long_trades) > 0:
            try:
                s1_rsi_values = [float(x) for x in long_trades['Symbol1 Entry RSI'] if str(x) != 'N/A']
                s2_rsi_values = [float(x) for x in long_trades['Symbol2 Entry RSI'] if str(x) != 'N/A']
                if s1_rsi_values and s2_rsi_values:
                    avg_rsi1_long = sum(s1_rsi_values) / len(s1_rsi_values)
                    avg_rsi2_long = sum(s2_rsi_values) / len(s2_rsi_values)
                    print(f"Long Trades - Avg Entry RSI: {symbol1}={avg_rsi1_long:.1f}, {symbol2}={avg_rsi2_long:.1f}")
            except (ValueError, TypeError):
                print(f"Long Trades - RSI data not available")
            
        if len(short_trades) > 0:
            try:
                s1_rsi_values = [float(x) for x in short_trades['Symbol1 Entry RSI'] if str(x) != 'N/A']
                s2_rsi_values = [float(x) for x in short_trades['Symbol2 Entry RSI'] if str(x) != 'N/A']
                if s1_rsi_values and s2_rsi_values:
                    avg_rsi1_short = sum(s1_rsi_values) / len(s1_rsi_values)
                    avg_rsi2_short = sum(s2_rsi_values) / len(s2_rsi_values)
                    print(f"Short Trades - Avg Entry RSI: {symbol1}={avg_rsi1_short:.1f}, {symbol2}={avg_rsi2_short:.1f}")
            except (ValueError, TypeError):
                print(f"Short Trades - RSI data not available")
        
        # Position Sizing Analysis
        try:
            avg_s1_lots = sum(report_df['Symbol1 Lots']) / len(report_df['Symbol1 Lots'])
            avg_s2_lots = sum(report_df['Symbol2 Lots']) / len(report_df['Symbol2 Lots'])
            hedge_ratios = [float(x) for x in report_df['Hedge Ratio'] if str(x) != 'N/A']
            avg_hedge_ratio = sum(hedge_ratios) / len(hedge_ratios) if hedge_ratios else 0
            
            print(f"\nPosition Sizing Analysis:")
            print(f"Average Lot Sizes: {symbol1}={avg_s1_lots:.2f}, {symbol2}={avg_s2_lots:.2f}")
            print(f"Average Hedge Ratio: {avg_hedge_ratio:.3f}")
        except (ValueError, TypeError, ZeroDivisionError):
            print(f"\nPosition Sizing Analysis: Data not available")
        
        # Pips vs USD Analysis
        total_pips = report_df['Total Pips'].sum()
        total_usd = report_df['Total P&L'].sum()
        print(f"\nPips vs USD Comparison:")
        print(f"Total Pips: {total_pips:.1f} | Total USD: ${total_usd:.2f}")
        print(f"Average USD per Pip: ${total_usd/total_pips:.2f}" if total_pips != 0 else "No pip data available")


In [49]:
def run_backtest(symbol1, symbol2):
    """
    Main function to run the backtest for a given pair of symbols.
    Now uses USD-based profit targets with comprehensive risk management.
    """
    print(f"\n----- Starting Backtest for {symbol1} / {symbol2} -----")
    
    # Get account balance for risk-based position sizing
    account_balance = get_account_balance()
    print(f"Account Balance: ${account_balance:,.2f}")
    
    # 1. Fetch Data
    df1 = get_historical_data(symbol1, TIMEFRAME, START_DATE, END_DATE)
    df2 = get_historical_data(symbol2, TIMEFRAME, START_DATE, END_DATE)

    if df1.empty or df2.empty:
        print(f"Could not fetch data for one of the symbols. Skipping pair.")
        return

    # 2. Combine data and calculate indicators
    df = pd.DataFrame(index=df1.index.union(df2.index))
    df['s1_close'] = df1['close']
    df['s2_close'] = df2['close']
    df.dropna(inplace=True) # Ensure we only have timestamps where both pairs have data

    df['s1_rsi'] = calculate_rsi(df['s1_close'], RSI_PERIOD)
    df['s2_rsi'] = calculate_rsi(df['s2_close'], RSI_PERIOD)
    df['s1_atr'] = calculate_atr(df1['high'], df1['low'], df1['close'], ATR_PERIOD)
    df['s2_atr'] = calculate_atr(df2['high'], df2['low'], df2['close'], ATR_PERIOD)
    df.dropna(inplace=True)

    print(f"Data prepared. Starting simulation with {len(df)} bars.")

    # 3. Simulation Loop
    trade_history = []
    trade_id_counter = 1
    in_trade = False
    current_trade = {}

    for i in range(len(df)):
        # --- CHECK EXIT CONDITIONS ---
        if in_trade:
            # Use the new USD-based exit condition check
            exit_result = check_exit_conditions(
                current_trade, 
                symbol1, 
                symbol2, 
                df.index[i], 
                df['s1_close'].iloc[i], 
                df['s2_close'].iloc[i]
            )
            
            if exit_result is not None:
                exit_reason, total_pnl, s1_pnl, s2_pnl = exit_result
                
                # Close the trade
                current_trade['exit_time'] = df.index[i]
                current_trade['s1_exit_price'] = df['s1_close'].iloc[i]
                current_trade['s2_exit_price'] = df['s2_close'].iloc[i]
                current_trade['exit_reason'] = exit_reason

                duration = (current_trade['exit_time'] - current_trade['entry_time']).total_seconds() / 3600

                # Calculate additional metrics for enhanced reporting
                s1_pips = calculate_pips(symbol1, current_trade['s1_entry_price'], current_trade['s1_exit_price'], current_trade['type'])
                s2_pips = calculate_pips(symbol2, current_trade['s2_entry_price'], current_trade['s2_exit_price'], current_trade['type'])
                
                trade_history.append({
                    # Basic Trade Info
                    "ID": current_trade['id'],
                    "Trade Type": current_trade['type'].upper(),
                    "Entry Time": current_trade['entry_time'],
                    "Exit Time": current_trade['exit_time'],
                    "Duration (hrs)": round(duration, 2),
                    "Exit Reason": exit_reason,
                    
                    # Symbol Information
                    "Symbol1": symbol1,
                    "Symbol2": symbol2,
                    
                    # Entry Conditions (stored during trade opening)
                    "Symbol1 Entry RSI": current_trade.get('s1_entry_rsi', 'N/A'),
                    "Symbol2 Entry RSI": current_trade.get('s2_entry_rsi', 'N/A'),
                    "Symbol1 Entry ATR": current_trade.get('s1_entry_atr', 'N/A'),
                    "Symbol2 Entry ATR": current_trade.get('s2_entry_atr', 'N/A'),
                    
                    # Price Information  
                    "Symbol1 Entry": current_trade['s1_entry_price'],
                    "Symbol1 Exit": current_trade['s1_exit_price'],
                    "Symbol2 Entry": current_trade['s2_entry_price'],
                    "Symbol2 Exit": current_trade['s2_exit_price'],
                    
                    # Position Sizing
                    "Symbol1 Lots": current_trade['s1_lots'],
                    "Symbol2 Lots": current_trade['s2_lots'],
                    "Hedge Ratio": current_trade.get('hedge_ratio', 'N/A'),
                    "Risk Amount USD": current_trade.get('risk_amount', 'N/A'),
                    "Account Balance": current_trade.get('account_balance', 'N/A'),
                    
                    # Performance Metrics
                    "Symbol1 Pips": round(s1_pips, 1),
                    "Symbol2 Pips": round(s2_pips, 1),
                    "Total Pips": round(s1_pips + s2_pips, 1),
                    "Symbol1 P&L": round(s1_pnl, 2),
                    "Symbol2 P&L": round(s2_pnl, 2),
                    "Total P&L": round(total_pnl, 2),
                    "Return %": round((total_pnl / current_trade.get('risk_amount', 100)) * 100, 2) if current_trade.get('risk_amount', 0) > 0 else 'N/A'
                })
                
                print(f"  Trade {current_trade['id']} closed: {exit_reason} | Total P&L: ${total_pnl:.2f} | Duration: {duration:.1f}h")
                
                in_trade = False
                current_trade = {}
                continue # Move to next bar after closing

        # --- CHECK ENTRY CONDITIONS ---
        if not in_trade:
            trade_type_to_open = None
            
            # Signal A: Short both pairs
            if df['s1_rsi'].iloc[i] > RSI_OVERBOUGHT and df['s2_rsi'].iloc[i] > RSI_OVERBOUGHT:
                trade_type_to_open = 'short'
            
            # Signal B: Long both pairs
            elif df['s1_rsi'].iloc[i] < RSI_OVERSOLD and df['s2_rsi'].iloc[i] < RSI_OVERSOLD:
                trade_type_to_open = 'long'

            if trade_type_to_open:
                # Calculate ATR values
                s1_atr_val = df['s1_atr'].iloc[i]
                s2_atr_val = df['s2_atr'].iloc[i]
                
                if s1_atr_val > 0 and s2_atr_val > 0:
                    # NEW: Use risk-based position sizing
                    s1_lot_size, s2_lot_size = calculate_risk_based_lots(
                        symbol1, symbol2, account_balance, s1_atr_val, s2_atr_val
                    )
                    
                    # Calculate additional metrics for storage
                    s1_pip_size = get_pip_size(symbol1)
                    s2_pip_size = get_pip_size(symbol2)
                    s1_atr_pips = s1_atr_val / s1_pip_size
                    s2_atr_pips = s2_atr_val / s2_pip_size
                    
                    # Calculate risk and hedge ratio
                    risk_usd = account_balance * (RISK_PER_TRADE_PCT / 100)
                    hedge_ratio = calculate_hedge_ratio(symbol1, symbol2, s1_atr_val, s2_atr_val)
                    
                    print(f"Entry Signal: {trade_type_to_open}")
                    print(f"  Risk per trade: ${risk_usd:.2f} ({RISK_PER_TRADE_PCT}% of balance)")
                    print(f"  {symbol1}: RSI={df['s1_rsi'].iloc[i]:.1f} | ATR={s1_atr_val:.5f} ({s1_atr_pips:.1f} pips) | Lot size: {s1_lot_size}")
                    print(f"  {symbol2}: RSI={df['s2_rsi'].iloc[i]:.1f} | ATR={s2_atr_val:.5f} ({s2_atr_pips:.1f} pips) | Lot size: {s2_lot_size}")
                    print(f"  Target: ${PROFIT_TARGET_USD} | Stop: ${STOP_LOSS_USD} | Max time: {MAX_TRADE_HOURS}h")
                    
                    # Open the trade with comprehensive information storage
                    in_trade = True
                    current_trade = {
                        # Basic trade info
                        'id': trade_id_counter,
                        'type': trade_type_to_open,
                        'entry_time': df.index[i],
                        
                        # Price and position info
                        's1_entry_price': df['s1_close'].iloc[i],
                        's2_entry_price': df['s2_close'].iloc[i],
                        's1_lots': s1_lot_size,
                        's2_lots': s2_lot_size,
                        
                        # Entry conditions for analysis
                        's1_entry_rsi': round(df['s1_rsi'].iloc[i], 1),
                        's2_entry_rsi': round(df['s2_rsi'].iloc[i], 1),
                        's1_entry_atr': round(s1_atr_val, 5),
                        's2_entry_atr': round(s2_atr_val, 5),
                        
                        # Risk management info
                        'hedge_ratio': round(hedge_ratio, 4),
                        'risk_amount': round(risk_usd, 2),
                        'account_balance': round(account_balance, 2)
                    }
                    trade_id_counter += 1

    # 4. Save Results
    if not trade_history:
        print("No trades were executed for this pair.")
    else:
        report_df = pd.DataFrame(trade_history)
        output_filename = f"{symbol1}_{symbol2}_backtest_report.csv"
        report_df.to_csv(output_filename, index=False)
        print(f"Backtest complete. Report saved to: {output_filename}")
        print("--- Summary ---")
        print(f"Total Trades: {len(report_df)}")
        print(f"Total P&L: ${report_df['Total P&L'].sum():.2f}")
        
        # Enhanced statistics and analysis
        profit_trades = report_df[report_df['Total P&L'] > 0]
        loss_trades = report_df[report_df['Total P&L'] < 0]
        
        print(f"\n--- Detailed Performance Analysis ---")
        print(f"Winning Trades: {len(profit_trades)} ({len(profit_trades)/len(report_df)*100:.1f}%)")
        print(f"Losing Trades: {len(loss_trades)} ({len(loss_trades)/len(report_df)*100:.1f}%)")
        print(f"Average Duration: {report_df['Duration (hrs)'].mean():.1f} hours")
        
        # P&L Statistics
        if len(profit_trades) > 0:
            print(f"Average Win: ${profit_trades['Total P&L'].mean():.2f}")
            print(f"Largest Win: ${profit_trades['Total P&L'].max():.2f}")
        if len(loss_trades) > 0:
            print(f"Average Loss: ${loss_trades['Total P&L'].mean():.2f}")
            print(f"Largest Loss: ${loss_trades['Total P&L'].min():.2f}")
        
        # Exit reason breakdown
        exit_reasons = report_df['Exit Reason'].value_counts()
        print(f"\nExit Reasons: {dict(exit_reasons)}")
        
        # Risk-Return Analysis
        try:
            returns_data = report_df[report_df['Return %'] != 'N/A']['Return %']
            if len(returns_data) > 0:
                numeric_returns = [float(x) for x in returns_data if str(x).replace('.', '').replace('-', '').isdigit()]
                if numeric_returns:
                    print(f"\nRisk-Return Analysis:")
                    print(f"Average Return per Trade: {sum(numeric_returns)/len(numeric_returns):.1f}%")
                    print(f"Best Return: {max(numeric_returns):.1f}%")
                    print(f"Worst Return: {min(numeric_returns):.1f}%")
        except (ValueError, TypeError):
            print(f"\nRisk-Return Analysis: Data not available")
        
        # Entry Condition Analysis
        long_trades = report_df[report_df['Trade Type'] == 'LONG']
        short_trades = report_df[report_df['Trade Type'] == 'SHORT']
        
        print(f"\nEntry Signal Analysis:")
        print(f"Long Trades: {len(long_trades)} | Short Trades: {len(short_trades)}")
        
        if len(long_trades) > 0:
            try:
                s1_rsi_values = [float(x) for x in long_trades['Symbol1 Entry RSI'] if str(x) != 'N/A']
                s2_rsi_values = [float(x) for x in long_trades['Symbol2 Entry RSI'] if str(x) != 'N/A']
                if s1_rsi_values and s2_rsi_values:
                    avg_rsi1_long = sum(s1_rsi_values) / len(s1_rsi_values)
                    avg_rsi2_long = sum(s2_rsi_values) / len(s2_rsi_values)
                    print(f"Long Trades - Avg Entry RSI: {symbol1}={avg_rsi1_long:.1f}, {symbol2}={avg_rsi2_long:.1f}")
            except (ValueError, TypeError):
                print(f"Long Trades - RSI data not available")
            
        if len(short_trades) > 0:
            try:
                s1_rsi_values = [float(x) for x in short_trades['Symbol1 Entry RSI'] if str(x) != 'N/A']
                s2_rsi_values = [float(x) for x in short_trades['Symbol2 Entry RSI'] if str(x) != 'N/A']
                if s1_rsi_values and s2_rsi_values:
                    avg_rsi1_short = sum(s1_rsi_values) / len(s1_rsi_values)
                    avg_rsi2_short = sum(s2_rsi_values) / len(s2_rsi_values)
                    print(f"Short Trades - Avg Entry RSI: {symbol1}={avg_rsi1_short:.1f}, {symbol2}={avg_rsi2_short:.1f}")
            except (ValueError, TypeError):
                print(f"Short Trades - RSI data not available")
        
        # Position Sizing Analysis
        try:
            avg_s1_lots = sum(report_df['Symbol1 Lots']) / len(report_df['Symbol1 Lots'])
            avg_s2_lots = sum(report_df['Symbol2 Lots']) / len(report_df['Symbol2 Lots'])
            hedge_ratios = [float(x) for x in report_df['Hedge Ratio'] if str(x) != 'N/A']
            avg_hedge_ratio = sum(hedge_ratios) / len(hedge_ratios) if hedge_ratios else 0
            
            print(f"\nPosition Sizing Analysis:")
            print(f"Average Lot Sizes: {symbol1}={avg_s1_lots:.2f}, {symbol2}={avg_s2_lots:.2f}")
            print(f"Average Hedge Ratio: {avg_hedge_ratio:.3f}")
        except (ValueError, TypeError, ZeroDivisionError):
            print(f"\nPosition Sizing Analysis: Data not available")
        
        # Pips vs USD Analysis
        total_pips = report_df['Total Pips'].sum()
        total_usd = report_df['Total P&L'].sum()
        print(f"\nPips vs USD Comparison:")
        print(f"Total Pips: {total_pips:.1f} | Total USD: ${total_usd:.2f}")
        print(f"Average USD per Pip: ${total_usd/total_pips:.2f}" if total_pips != 0 else "No pip data available")


In [50]:
def run_backtest(symbol1, symbol2):
    """
    Main function to run the backtest for a given pair of symbols.
    Now uses USD-based profit targets with comprehensive risk management.
    """
    print(f"\n----- Starting Backtest for {symbol1} / {symbol2} -----")
    
    # Get account balance for risk-based position sizing
    account_balance = get_account_balance()
    print(f"Account Balance: ${account_balance:,.2f}")
    
    # 1. Fetch Data
    df1 = get_historical_data(symbol1, TIMEFRAME, START_DATE, END_DATE)
    df2 = get_historical_data(symbol2, TIMEFRAME, START_DATE, END_DATE)

    if df1.empty or df2.empty:
        print(f"Could not fetch data for one of the symbols. Skipping pair.")
        return

    # 2. Combine data and calculate indicators
    df = pd.DataFrame(index=df1.index.union(df2.index))
    df['s1_close'] = df1['close']
    df['s2_close'] = df2['close']
    df.dropna(inplace=True) # Ensure we only have timestamps where both pairs have data

    df['s1_rsi'] = calculate_rsi(df['s1_close'], RSI_PERIOD)
    df['s2_rsi'] = calculate_rsi(df['s2_close'], RSI_PERIOD)
    df['s1_atr'] = calculate_atr(df1['high'], df1['low'], df1['close'], ATR_PERIOD)
    df['s2_atr'] = calculate_atr(df2['high'], df2['low'], df2['close'], ATR_PERIOD)
    df.dropna(inplace=True)

    print(f"Data prepared. Starting simulation with {len(df)} bars.")

    # 3. Simulation Loop
    trade_history = []
    trade_id_counter = 1
    in_trade = False
    current_trade = {}

    for i in range(len(df)):
        # --- CHECK EXIT CONDITIONS ---
        if in_trade:
            # Use the new USD-based exit condition check
            exit_result = check_exit_conditions(
                current_trade, 
                symbol1, 
                symbol2, 
                df.index[i], 
                df['s1_close'].iloc[i], 
                df['s2_close'].iloc[i]
            )
            
            if exit_result is not None:
                exit_reason, total_pnl, s1_pnl, s2_pnl = exit_result
                
                # Close the trade
                current_trade['exit_time'] = df.index[i]
                current_trade['s1_exit_price'] = df['s1_close'].iloc[i]
                current_trade['s2_exit_price'] = df['s2_close'].iloc[i]
                current_trade['exit_reason'] = exit_reason

                duration = (current_trade['exit_time'] - current_trade['entry_time']).total_seconds() / 3600

                # Calculate additional metrics for enhanced reporting
                s1_pips = calculate_pips(symbol1, current_trade['s1_entry_price'], current_trade['s1_exit_price'], current_trade['type'])
                s2_pips = calculate_pips(symbol2, current_trade['s2_entry_price'], current_trade['s2_exit_price'], current_trade['type'])
                
                trade_history.append({
                    # Basic Trade Info
                    "ID": current_trade['id'],
                    "Trade Type": current_trade['type'].upper(),
                    "Entry Time": current_trade['entry_time'],
                    "Exit Time": current_trade['exit_time'],
                    "Duration (hrs)": round(duration, 2),
                    "Exit Reason": exit_reason,
                    
                    # Symbol Information
                    "Symbol1": symbol1,
                    "Symbol2": symbol2,
                    
                    # Entry Conditions (stored during trade opening)
                    "Symbol1 Entry RSI": current_trade.get('s1_entry_rsi', 'N/A'),
                    "Symbol2 Entry RSI": current_trade.get('s2_entry_rsi', 'N/A'),
                    "Symbol1 Entry ATR": current_trade.get('s1_entry_atr', 'N/A'),
                    "Symbol2 Entry ATR": current_trade.get('s2_entry_atr', 'N/A'),
                    
                    # Price Information  
                    "Symbol1 Entry": current_trade['s1_entry_price'],
                    "Symbol1 Exit": current_trade['s1_exit_price'],
                    "Symbol2 Entry": current_trade['s2_entry_price'],
                    "Symbol2 Exit": current_trade['s2_exit_price'],
                    
                    # Position Sizing
                    "Symbol1 Lots": current_trade['s1_lots'],
                    "Symbol2 Lots": current_trade['s2_lots'],
                    "Hedge Ratio": current_trade.get('hedge_ratio', 'N/A'),
                    "Risk Amount USD": current_trade.get('risk_amount', 'N/A'),
                    "Account Balance": current_trade.get('account_balance', 'N/A'),
                    
                    # Performance Metrics
                    "Symbol1 Pips": round(s1_pips, 1),
                    "Symbol2 Pips": round(s2_pips, 1),
                    "Total Pips": round(s1_pips + s2_pips, 1),
                    "Symbol1 P&L": round(s1_pnl, 2),
                    "Symbol2 P&L": round(s2_pnl, 2),
                    "Total P&L": round(total_pnl, 2),
                    "Return %": round((total_pnl / current_trade.get('risk_amount', 100)) * 100, 2) if current_trade.get('risk_amount', 0) > 0 else 'N/A'
                })
                
                print(f"  Trade {current_trade['id']} closed: {exit_reason} | Total P&L: ${total_pnl:.2f} | Duration: {duration:.1f}h")
                
                in_trade = False
                current_trade = {}
                continue # Move to next bar after closing

        # --- CHECK ENTRY CONDITIONS ---
        if not in_trade:
            trade_type_to_open = None
            
            # Signal A: Short both pairs
            if df['s1_rsi'].iloc[i] > RSI_OVERBOUGHT and df['s2_rsi'].iloc[i] > RSI_OVERBOUGHT:
                trade_type_to_open = 'short'
            
            # Signal B: Long both pairs
            elif df['s1_rsi'].iloc[i] < RSI_OVERSOLD and df['s2_rsi'].iloc[i] < RSI_OVERSOLD:
                trade_type_to_open = 'long'

            if trade_type_to_open:
                # Calculate ATR values
                s1_atr_val = df['s1_atr'].iloc[i]
                s2_atr_val = df['s2_atr'].iloc[i]
                
                if s1_atr_val > 0 and s2_atr_val > 0:
                    # NEW: Use risk-based position sizing
                    s1_lot_size, s2_lot_size = calculate_risk_based_lots(
                        symbol1, symbol2, account_balance, s1_atr_val, s2_atr_val
                    )
                    
                    # Calculate additional metrics for storage
                    s1_pip_size = get_pip_size(symbol1)
                    s2_pip_size = get_pip_size(symbol2)
                    s1_atr_pips = s1_atr_val / s1_pip_size
                    s2_atr_pips = s2_atr_val / s2_pip_size
                    
                    # Calculate risk and hedge ratio
                    risk_usd = account_balance * (RISK_PER_TRADE_PCT / 100)
                    hedge_ratio = calculate_hedge_ratio(symbol1, symbol2, s1_atr_val, s2_atr_val)
                    
                    print(f"Entry Signal: {trade_type_to_open}")
                    print(f"  Risk per trade: ${risk_usd:.2f} ({RISK_PER_TRADE_PCT}% of balance)")
                    print(f"  {symbol1}: RSI={df['s1_rsi'].iloc[i]:.1f} | ATR={s1_atr_val:.5f} ({s1_atr_pips:.1f} pips) | Lot size: {s1_lot_size}")
                    print(f"  {symbol2}: RSI={df['s2_rsi'].iloc[i]:.1f} | ATR={s2_atr_val:.5f} ({s2_atr_pips:.1f} pips) | Lot size: {s2_lot_size}")
                    print(f"  Target: ${PROFIT_TARGET_USD} | Stop: ${STOP_LOSS_USD} | Max time: {MAX_TRADE_HOURS}h")
                    
                    # Open the trade with comprehensive information storage
                    in_trade = True
                    current_trade = {
                        # Basic trade info
                        'id': trade_id_counter,
                        'type': trade_type_to_open,
                        'entry_time': df.index[i],
                        
                        # Price and position info
                        's1_entry_price': df['s1_close'].iloc[i],
                        's2_entry_price': df['s2_close'].iloc[i],
                        's1_lots': s1_lot_size,
                        's2_lots': s2_lot_size,
                        
                        # Entry conditions for analysis
                        's1_entry_rsi': round(df['s1_rsi'].iloc[i], 1),
                        's2_entry_rsi': round(df['s2_rsi'].iloc[i], 1),
                        's1_entry_atr': round(s1_atr_val, 5),
                        's2_entry_atr': round(s2_atr_val, 5),
                        
                        # Risk management info
                        'hedge_ratio': round(hedge_ratio, 4),
                        'risk_amount': round(risk_usd, 2),
                        'account_balance': round(account_balance, 2)
                    }
                    trade_id_counter += 1

    # 4. Save Results
    if not trade_history:
        print("No trades were executed for this pair.")
    else:
        report_df = pd.DataFrame(trade_history)
        output_filename = f"{symbol1}_{symbol2}_backtest_report.csv"
        report_df.to_csv(output_filename, index=False)
        print(f"Backtest complete. Report saved to: {output_filename}")
        print("--- Summary ---")
        print(f"Total Trades: {len(report_df)}")
        print(f"Total P&L: ${report_df['Total P&L'].sum():.2f}")
        
        # Enhanced statistics and analysis
        profit_trades = report_df[report_df['Total P&L'] > 0]
        loss_trades = report_df[report_df['Total P&L'] < 0]
        
        print(f"\n--- Detailed Performance Analysis ---")
        print(f"Winning Trades: {len(profit_trades)} ({len(profit_trades)/len(report_df)*100:.1f}%)")
        print(f"Losing Trades: {len(loss_trades)} ({len(loss_trades)/len(report_df)*100:.1f}%)")
        print(f"Average Duration: {report_df['Duration (hrs)'].mean():.1f} hours")
        
        # P&L Statistics
        if len(profit_trades) > 0:
            print(f"Average Win: ${profit_trades['Total P&L'].mean():.2f}")
            print(f"Largest Win: ${profit_trades['Total P&L'].max():.2f}")
        if len(loss_trades) > 0:
            print(f"Average Loss: ${loss_trades['Total P&L'].mean():.2f}")
            print(f"Largest Loss: ${loss_trades['Total P&L'].min():.2f}")
        
        # Exit reason breakdown
        exit_reasons = report_df['Exit Reason'].value_counts()
        print(f"\nExit Reasons: {dict(exit_reasons)}")
        
        # Risk-Return Analysis
        try:
            returns_data = report_df[report_df['Return %'] != 'N/A']['Return %']
            if len(returns_data) > 0:
                numeric_returns = [float(x) for x in returns_data if str(x).replace('.', '').replace('-', '').isdigit()]
                if numeric_returns:
                    print(f"\nRisk-Return Analysis:")
                    print(f"Average Return per Trade: {sum(numeric_returns)/len(numeric_returns):.1f}%")
                    print(f"Best Return: {max(numeric_returns):.1f}%")
                    print(f"Worst Return: {min(numeric_returns):.1f}%")
        except (ValueError, TypeError):
            print(f"\nRisk-Return Analysis: Data not available")
        
        # Entry Condition Analysis
        long_trades = report_df[report_df['Trade Type'] == 'LONG']
        short_trades = report_df[report_df['Trade Type'] == 'SHORT']
        
        print(f"\nEntry Signal Analysis:")
        print(f"Long Trades: {len(long_trades)} | Short Trades: {len(short_trades)}")
        
        if len(long_trades) > 0:
            try:
                s1_rsi_values = [float(x) for x in long_trades['Symbol1 Entry RSI'] if str(x) != 'N/A']
                s2_rsi_values = [float(x) for x in long_trades['Symbol2 Entry RSI'] if str(x) != 'N/A']
                if s1_rsi_values and s2_rsi_values:
                    avg_rsi1_long = sum(s1_rsi_values) / len(s1_rsi_values)
                    avg_rsi2_long = sum(s2_rsi_values) / len(s2_rsi_values)
                    print(f"Long Trades - Avg Entry RSI: {symbol1}={avg_rsi1_long:.1f}, {symbol2}={avg_rsi2_long:.1f}")
            except (ValueError, TypeError):
                print(f"Long Trades - RSI data not available")
            
        if len(short_trades) > 0:
            try:
                s1_rsi_values = [float(x) for x in short_trades['Symbol1 Entry RSI'] if str(x) != 'N/A']
                s2_rsi_values = [float(x) for x in short_trades['Symbol2 Entry RSI'] if str(x) != 'N/A']
                if s1_rsi_values and s2_rsi_values:
                    avg_rsi1_short = sum(s1_rsi_values) / len(s1_rsi_values)
                    avg_rsi2_short = sum(s2_rsi_values) / len(s2_rsi_values)
                    print(f"Short Trades - Avg Entry RSI: {symbol1}={avg_rsi1_short:.1f}, {symbol2}={avg_rsi2_short:.1f}")
            except (ValueError, TypeError):
                print(f"Short Trades - RSI data not available")
        
        # Position Sizing Analysis
        try:
            avg_s1_lots = sum(report_df['Symbol1 Lots']) / len(report_df['Symbol1 Lots'])
            avg_s2_lots = sum(report_df['Symbol2 Lots']) / len(report_df['Symbol2 Lots'])
            hedge_ratios = [float(x) for x in report_df['Hedge Ratio'] if str(x) != 'N/A']
            avg_hedge_ratio = sum(hedge_ratios) / len(hedge_ratios) if hedge_ratios else 0
            
            print(f"\nPosition Sizing Analysis:")
            print(f"Average Lot Sizes: {symbol1}={avg_s1_lots:.2f}, {symbol2}={avg_s2_lots:.2f}")
            print(f"Average Hedge Ratio: {avg_hedge_ratio:.3f}")
        except (ValueError, TypeError, ZeroDivisionError):
            print(f"\nPosition Sizing Analysis: Data not available")
        
        # Pips vs USD Analysis
        total_pips = report_df['Total Pips'].sum()
        total_usd = report_df['Total P&L'].sum()
        print(f"\nPips vs USD Comparison:")
        print(f"Total Pips: {total_pips:.1f} | Total USD: ${total_usd:.2f}")
        print(f"Average USD per Pip: ${total_usd/total_pips:.2f}" if total_pips != 0 else "No pip data available")


In [51]:
def run_backtest(symbol1, symbol2):
    """
    Main function to run the backtest for a given pair of symbols.
    Now uses USD-based profit targets with comprehensive risk management.
    """
    print(f"\n----- Starting Backtest for {symbol1} / {symbol2} -----")
    
    # Get account balance for risk-based position sizing
    account_balance = get_account_balance()
    print(f"Account Balance: ${account_balance:,.2f}")
    
    # 1. Fetch Data
    df1 = get_historical_data(symbol1, TIMEFRAME, START_DATE, END_DATE)
    df2 = get_historical_data(symbol2, TIMEFRAME, START_DATE, END_DATE)

    if df1.empty or df2.empty:
        print(f"Could not fetch data for one of the symbols. Skipping pair.")
        return

    # 2. Combine data and calculate indicators
    df = pd.DataFrame(index=df1.index.union(df2.index))
    df['s1_close'] = df1['close']
    df['s2_close'] = df2['close']
    df.dropna(inplace=True) # Ensure we only have timestamps where both pairs have data

    df['s1_rsi'] = calculate_rsi(df['s1_close'], RSI_PERIOD)
    df['s2_rsi'] = calculate_rsi(df['s2_close'], RSI_PERIOD)
    df['s1_atr'] = calculate_atr(df1['high'], df1['low'], df1['close'], ATR_PERIOD)
    df['s2_atr'] = calculate_atr(df2['high'], df2['low'], df2['close'], ATR_PERIOD)
    df.dropna(inplace=True)

    print(f"Data prepared. Starting simulation with {len(df)} bars.")

    # 3. Simulation Loop
    trade_history = []
    trade_id_counter = 1
    in_trade = False
    current_trade = {}

    for i in range(len(df)):
        # --- CHECK EXIT CONDITIONS ---
        if in_trade:
            # Use the new USD-based exit condition check
            exit_result = check_exit_conditions(
                current_trade, 
                symbol1, 
                symbol2, 
                df.index[i], 
                df['s1_close'].iloc[i], 
                df['s2_close'].iloc[i]
            )
            
            if exit_result is not None:
                exit_reason, total_pnl, s1_pnl, s2_pnl = exit_result
                
                # Close the trade
                current_trade['exit_time'] = df.index[i]
                current_trade['s1_exit_price'] = df['s1_close'].iloc[i]
                current_trade['s2_exit_price'] = df['s2_close'].iloc[i]
                current_trade['exit_reason'] = exit_reason

                duration = (current_trade['exit_time'] - current_trade['entry_time']).total_seconds() / 3600

                # Calculate additional metrics for enhanced reporting
                s1_pips = calculate_pips(symbol1, current_trade['s1_entry_price'], current_trade['s1_exit_price'], current_trade['type'])
                s2_pips = calculate_pips(symbol2, current_trade['s2_entry_price'], current_trade['s2_exit_price'], current_trade['type'])
                
                trade_history.append({
                    # Basic Trade Info
                    "ID": current_trade['id'],
                    "Trade Type": current_trade['type'].upper(),
                    "Entry Time": current_trade['entry_time'],
                    "Exit Time": current_trade['exit_time'],
                    "Duration (hrs)": round(duration, 2),
                    "Exit Reason": exit_reason,
                    
                    # Symbol Information
                    "Symbol1": symbol1,
                    "Symbol2": symbol2,
                    
                    # Entry Conditions (stored during trade opening)
                    "Symbol1 Entry RSI": current_trade.get('s1_entry_rsi', 'N/A'),
                    "Symbol2 Entry RSI": current_trade.get('s2_entry_rsi', 'N/A'),
                    "Symbol1 Entry ATR": current_trade.get('s1_entry_atr', 'N/A'),
                    "Symbol2 Entry ATR": current_trade.get('s2_entry_atr', 'N/A'),
                    
                    # Price Information  
                    "Symbol1 Entry": current_trade['s1_entry_price'],
                    "Symbol1 Exit": current_trade['s1_exit_price'],
                    "Symbol2 Entry": current_trade['s2_entry_price'],
                    "Symbol2 Exit": current_trade['s2_exit_price'],
                    
                    # Position Sizing
                    "Symbol1 Lots": current_trade['s1_lots'],
                    "Symbol2 Lots": current_trade['s2_lots'],
                    "Hedge Ratio": current_trade.get('hedge_ratio', 'N/A'),
                    "Risk Amount USD": current_trade.get('risk_amount', 'N/A'),
                    "Account Balance": current_trade.get('account_balance', 'N/A'),
                    
                    # Performance Metrics
                    "Symbol1 Pips": round(s1_pips, 1),
                    "Symbol2 Pips": round(s2_pips, 1),
                    "Total Pips": round(s1_pips + s2_pips, 1),
                    "Symbol1 P&L": round(s1_pnl, 2),
                    "Symbol2 P&L": round(s2_pnl, 2),
                    "Total P&L": round(total_pnl, 2),
                    "Return %": round((total_pnl / current_trade.get('risk_amount', 100)) * 100, 2) if current_trade.get('risk_amount', 0) > 0 else 'N/A'
                })
                
                print(f"  Trade {current_trade['id']} closed: {exit_reason} | Total P&L: ${total_pnl:.2f} | Duration: {duration:.1f}h")
                
                in_trade = False
                current_trade = {}
                continue # Move to next bar after closing

        # --- CHECK ENTRY CONDITIONS ---
        if not in_trade:
            trade_type_to_open = None
            
            # Signal A: Short both pairs
            if df['s1_rsi'].iloc[i] > RSI_OVERBOUGHT and df['s2_rsi'].iloc[i] > RSI_OVERBOUGHT:
                trade_type_to_open = 'short'
            
            # Signal B: Long both pairs
            elif df['s1_rsi'].iloc[i] < RSI_OVERSOLD and df['s2_rsi'].iloc[i] < RSI_OVERSOLD:
                trade_type_to_open = 'long'

            if trade_type_to_open:
                # Calculate ATR values
                s1_atr_val = df['s1_atr'].iloc[i]
                s2_atr_val = df['s2_atr'].iloc[i]
                
                if s1_atr_val > 0 and s2_atr_val > 0:
                    # NEW: Use risk-based position sizing
                    s1_lot_size, s2_lot_size = calculate_risk_based_lots(
                        symbol1, symbol2, account_balance, s1_atr_val, s2_atr_val
                    )
                    
                    # Calculate additional metrics for storage
                    s1_pip_size = get_pip_size(symbol1)
                    s2_pip_size = get_pip_size(symbol2)
                    s1_atr_pips = s1_atr_val / s1_pip_size
                    s2_atr_pips = s2_atr_val / s2_pip_size
                    
                    # Calculate risk and hedge ratio
                    risk_usd = account_balance * (RISK_PER_TRADE_PCT / 100)
                    hedge_ratio = calculate_hedge_ratio(symbol1, symbol2, s1_atr_val, s2_atr_val)
                    
                    print(f"Entry Signal: {trade_type_to_open}")
                    print(f"  Risk per trade: ${risk_usd:.2f} ({RISK_PER_TRADE_PCT}% of balance)")
                    print(f"  {symbol1}: RSI={df['s1_rsi'].iloc[i]:.1f} | ATR={s1_atr_val:.5f} ({s1_atr_pips:.1f} pips) | Lot size: {s1_lot_size}")
                    print(f"  {symbol2}: RSI={df['s2_rsi'].iloc[i]:.1f} | ATR={s2_atr_val:.5f} ({s2_atr_pips:.1f} pips) | Lot size: {s2_lot_size}")
                    print(f"  Target: ${PROFIT_TARGET_USD} | Stop: ${STOP_LOSS_USD} | Max time: {MAX_TRADE_HOURS}h")
                    
                    # Open the trade with comprehensive information storage
                    in_trade = True
                    current_trade = {
                        # Basic trade info
                        'id': trade_id_counter,
                        'type': trade_type_to_open,
                        'entry_time': df.index[i],
                        
                        # Price and position info
                        's1_entry_price': df['s1_close'].iloc[i],
                        's2_entry_price': df['s2_close'].iloc[i],
                        's1_lots': s1_lot_size,
                        's2_lots': s2_lot_size,
                        
                        # Entry conditions for analysis
                        's1_entry_rsi': round(df['s1_rsi'].iloc[i], 1),
                        's2_entry_rsi': round(df['s2_rsi'].iloc[i], 1),
                        's1_entry_atr': round(s1_atr_val, 5),
                        's2_entry_atr': round(s2_atr_val, 5),
                        
                        # Risk management info
                        'hedge_ratio': round(hedge_ratio, 4),
                        'risk_amount': round(risk_usd, 2),
                        'account_balance': round(account_balance, 2)
                    }
                    trade_id_counter += 1

    # 4. Save Results
    if not trade_history:
        print("No trades were executed for this pair.")
    else:
        report_df = pd.DataFrame(trade_history)
        output_filename = f"{symbol1}_{symbol2}_backtest_report.csv"
        report_df.to_csv(output_filename, index=False)
        print(f"Backtest complete. Report saved to: {output_filename}")
        print("--- Summary ---")
        print(f"Total Trades: {len(report_df)}")
        print(f"Total P&L: ${report_df['Total P&L'].sum():.2f}")
        
        # Enhanced statistics and analysis
        profit_trades = report_df[report_df['Total P&L'] > 0]
        loss_trades = report_df[report_df['Total P&L'] < 0]
        
        print(f"\n--- Detailed Performance Analysis ---")
        print(f"Winning Trades: {len(profit_trades)} ({len(profit_trades)/len(report_df)*100:.1f}%)")
        print(f"Losing Trades: {len(loss_trades)} ({len(loss_trades)/len(report_df)*100:.1f}%)")
        print(f"Average Duration: {report_df['Duration (hrs)'].mean():.1f} hours")
        
        # P&L Statistics
        if len(profit_trades) > 0:
            print(f"Average Win: ${profit_trades['Total P&L'].mean():.2f}")
            print(f"Largest Win: ${profit_trades['Total P&L'].max():.2f}")
        if len(loss_trades) > 0:
            print(f"Average Loss: ${loss_trades['Total P&L'].mean():.2f}")
            print(f"Largest Loss: ${loss_trades['Total P&L'].min():.2f}")
        
        # Exit reason breakdown
        exit_reasons = report_df['Exit Reason'].value_counts()
        print(f"\nExit Reasons: {dict(exit_reasons)}")
        
        # Risk-Return Analysis
        try:
            returns_data = report_df[report_df['Return %'] != 'N/A']['Return %']
            if len(returns_data) > 0:
                numeric_returns = [float(x) for x in returns_data if str(x).replace('.', '').replace('-', '').isdigit()]
                if numeric_returns:
                    print(f"\nRisk-Return Analysis:")
                    print(f"Average Return per Trade: {sum(numeric_returns)/len(numeric_returns):.1f}%")
                    print(f"Best Return: {max(numeric_returns):.1f}%")
                    print(f"Worst Return: {min(numeric_returns):.1f}%")
        except (ValueError, TypeError):
            print(f"\nRisk-Return Analysis: Data not available")
        
        # Entry Condition Analysis
        long_trades = report_df[report_df['Trade Type'] == 'LONG']
        short_trades = report_df[report_df['Trade Type'] == 'SHORT']
        
        print(f"\nEntry Signal Analysis:")
        print(f"Long Trades: {len(long_trades)} | Short Trades: {len(short_trades)}")
        
        if len(long_trades) > 0:
            try:
                s1_rsi_values = [float(x) for x in long_trades['Symbol1 Entry RSI'] if str(x) != 'N/A']
                s2_rsi_values = [float(x) for x in long_trades['Symbol2 Entry RSI'] if str(x) != 'N/A']
                if s1_rsi_values and s2_rsi_values:
                    avg_rsi1_long = sum(s1_rsi_values) / len(s1_rsi_values)
                    avg_rsi2_long = sum(s2_rsi_values) / len(s2_rsi_values)
                    print(f"Long Trades - Avg Entry RSI: {symbol1}={avg_rsi1_long:.1f}, {symbol2}={avg_rsi2_long:.1f}")
            except (ValueError, TypeError):
                print(f"Long Trades - RSI data not available")
            
        if len(short_trades) > 0:
            try:
                s1_rsi_values = [float(x) for x in short_trades['Symbol1 Entry RSI'] if str(x) != 'N/A']
                s2_rsi_values = [float(x) for x in short_trades['Symbol2 Entry RSI'] if str(x) != 'N/A']
                if s1_rsi_values and s2_rsi_values:
                    avg_rsi1_short = sum(s1_rsi_values) / len(s1_rsi_values)
                    avg_rsi2_short = sum(s2_rsi_values) / len(s2_rsi_values)
                    print(f"Short Trades - Avg Entry RSI: {symbol1}={avg_rsi1_short:.1f}, {symbol2}={avg_rsi2_short:.1f}")
            except (ValueError, TypeError):
                print(f"Short Trades - RSI data not available")
        
        # Position Sizing Analysis
        try:
            avg_s1_lots = sum(report_df['Symbol1 Lots']) / len(report_df['Symbol1 Lots'])
            avg_s2_lots = sum(report_df['Symbol2 Lots']) / len(report_df['Symbol2 Lots'])
            hedge_ratios = [float(x) for x in report_df['Hedge Ratio'] if str(x) != 'N/A']
            avg_hedge_ratio = sum(hedge_ratios) / len(hedge_ratios) if hedge_ratios else 0
            
            print(f"\nPosition Sizing Analysis:")
            print(f"Average Lot Sizes: {symbol1}={avg_s1_lots:.2f}, {symbol2}={avg_s2_lots:.2f}")
            print(f"Average Hedge Ratio: {avg_hedge_ratio:.3f}")
        except (ValueError, TypeError, ZeroDivisionError):
            print(f"\nPosition Sizing Analysis: Data not available")
        
        # Pips vs USD Analysis
        total_pips = report_df['Total Pips'].sum()
        total_usd = report_df['Total P&L'].sum()
        print(f"\nPips vs USD Comparison:")
        print(f"Total Pips: {total_pips:.1f} | Total USD: ${total_usd:.2f}")
        print(f"Average USD per Pip: ${total_usd/total_pips:.2f}" if total_pips != 0 else "No pip data available")


In [52]:
def get_historical_data(symbol, timeframe, start_date, end_date):
    """Fetches historical data from MT5."""
    rates = mt5.copy_rates_range(symbol, timeframe, start_date, end_date)
    if rates is None or len(rates) == 0:
        print(f"Failed to get rates for {symbol}, error code = {mt5.last_error()}")
        return pd.DataFrame()
    
    data = pd.DataFrame(rates)
    data['time'] = pd.to_datetime(data['time'], unit='s')
    data.set_index('time', inplace=True)
    return data

def get_pip_size(symbol):
    """
    Determines the pip size for any symbol type with comprehensive support.
    Returns the pip size (not point size) for proper pip calculations.
    """
    symbol = symbol.upper()
    
    # Get symbol info from MT5
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"Warning: Could not get symbol info for {symbol}, using default pip size")
        return 0.0001
    
    # Precious metals
    if symbol.startswith('XAU'):  # Gold (XAUUSD, XAUGBP, etc.)
        return 0.1
    elif symbol.startswith('XAG'):  # Silver (XAGUSD, XAGGBP, etc.)
        return 0.001
    elif symbol.startswith('XPD') or symbol.startswith('XPT'):  # Palladium/Platinum
        return 0.1
    
    # JPY pairs - pip is always 0.01 regardless of digits
    elif 'JPY' in symbol:
        return 0.01
    
    # Forex pairs
    else:
        # Use point size to determine pip size
        point = symbol_info.point
        digits = symbol_info.digits
        
        if digits == 5 or digits == 3:  # 5-digit or 3-digit broker
            return point * 10  # Pip is 10x the point
        elif digits == 4 or digits == 2:  # 4-digit or 2-digit broker
            return point  # Pip equals point
        else:
            # Fallback based on point size
            if point == 0.00001:
                return 0.0001
            elif point == 0.001:
                return 0.01
            else:
                return 0.0001

def get_symbol_lot_info(symbol):
    """
    Gets lot size constraints for a symbol from MT5.
    Returns (min_lot, max_lot, lot_step)
    """
    symbol_info = mt5.symbol_info(symbol)
    if symbol_info is None:
        print(f"Warning: Could not get symbol info for {symbol}, using defaults")
        return 0.01, 100.0, 0.01
    
    return symbol_info.volume_min, symbol_info.volume_max, symbol_info.volume_step

def normalize_lot_size(symbol, lot_size):
    """
    Ensures lot size conforms to broker requirements and safety bounds.
    """
    min_lot, max_lot, lot_step = get_symbol_lot_info(symbol)
    
    # Apply safety bounds (prevent extreme lot sizes)
    SAFETY_MIN_LOT = max(0.01, min_lot)
    SAFETY_MAX_LOT = min(10.0, max_lot)
    
    # Clamp to safety bounds
    lot_size = max(SAFETY_MIN_LOT, min(SAFETY_MAX_LOT, lot_size))
    
    # Round to nearest step
    if lot_step > 0:
        lot_size = round(lot_size / lot_step) * lot_step
    
    # Ensure minimum lot size
    lot_size = max(min_lot, lot_size)
    
    return round(lot_size, 4)

def calculate_hedge_ratio(symbol1, symbol2, atr1, atr2):
    """
    Calculates proper hedge ratio using ATR normalized to pips.
    Returns the ratio to apply to symbol2 lot size.
    """
    # Convert ATR to pips for both symbols
    pip_size1 = get_pip_size(symbol1)
    pip_size2 = get_pip_size(symbol2)
    
    atr1_pips = atr1 / pip_size1
    atr2_pips = atr2 / pip_size2
    
    # Validate ATR values
    if atr1_pips <= 0 or atr2_pips <= 0:
        print(f"Warning: Invalid ATR values for {symbol1}/{symbol2}, using 1:1 ratio")
        return 1.0
    
    # Calculate volatility ratio
    volatility_ratio = atr1_pips / atr2_pips
    
    # Apply safety bounds to prevent extreme ratios
    MAX_RATIO = 5.0
    MIN_RATIO = 0.2
    
    if volatility_ratio > MAX_RATIO:
        print(f"Warning: Extreme ratio {volatility_ratio:.2f} for {symbol1}/{symbol2}, capped at {MAX_RATIO}")
        volatility_ratio = MAX_RATIO
    elif volatility_ratio < MIN_RATIO:
        print(f"Warning: Extreme ratio {volatility_ratio:.2f} for {symbol1}/{symbol2}, raised to {MIN_RATIO}")
        volatility_ratio = MIN_RATIO
    
    return volatility_ratio

def calculate_pips(symbol, entry_price, current_price, trade_type):
    """Calculates the profit/loss of a trade in pips."""
    pip_size = get_pip_size(symbol)
    if trade_type == 'long':
        pips = (current_price - entry_price) / pip_size
    elif trade_type == 'short':
        pips = (entry_price - current_price) / pip_size
    else:
        pips = 0
    return pips

def calculate_pnl_usd(symbol, trade_type, lot_size, entry_price, exit_price):
    """Calculates the final P&L of a trade in USD using MT5's calculator."""
    mt5_trade_type = mt5.ORDER_TYPE_BUY if trade_type == 'long' else mt5.ORDER_TYPE_SELL
    
    result = mt5.order_calc_profit(
        mt5_trade_type,
        symbol,
        lot_size,
        entry_price,
        exit_price
    )
    
    if result is None:
        print(f"Failed to calculate P&L for {symbol}, error: {mt5.last_error()}")
        return 0.0
    return result

def get_account_balance():
    """Gets current account balance for risk calculations."""
    account_info = mt5.account_info()
    if account_info:
        return account_info.balance
    else:
        print("Warning: Could not get account balance, using default 10000")
        return 10000.0

def calculate_simple_lots(symbol1, symbol2, atr1, atr2):
    """
    Calculates lot sizes using simple hedge ratio approach.
    Symbol1 uses BASE_LOT_SIZE, Symbol2 is adjusted using hedge ratio.
    """
    # Use base lot size for symbol1
    s1_lot_size = BASE_LOT_SIZE
    
    # Calculate hedge ratio and lot size for symbol2
    hedge_ratio = calculate_hedge_ratio(symbol1, symbol2, atr1, atr2)
    s2_lot_size = s1_lot_size * hedge_ratio
    
    # Normalize lot sizes
    s1_lots = normalize_lot_size(symbol1, s1_lot_size)
    s2_lots = normalize_lot_size(symbol2, s2_lot_size)
    
    return s1_lots, s2_lots

def check_exit_conditions(current_trade, symbol1, symbol2, current_time, s1_price, s2_price):
    """
    Checks all exit conditions: profit target, stop loss, and time limit.
    Returns exit reason or None if no exit.
    """
    # Calculate current USD P&L for both positions
    s1_pnl = calculate_pnl_usd(
        symbol1, 
        current_trade['type'], 
        current_trade['s1_lots'], 
        current_trade['s1_entry_price'], 
        s1_price
    )
    
    s2_pnl = calculate_pnl_usd(
        symbol2, 
        current_trade['type'], 
        current_trade['s2_lots'], 
        current_trade['s2_entry_price'], 
        s2_price
    )
    
    total_pnl = s1_pnl + s2_pnl
    
    # Check profit target
    if total_pnl >= PROFIT_TARGET_USD:
        return "PROFIT_TARGET", total_pnl, s1_pnl, s2_pnl
    
    # Check stop loss
    if total_pnl <= STOP_LOSS_USD:
        return "STOP_LOSS", total_pnl, s1_pnl, s2_pnl
    
    # Check time limit
    duration_hours = (current_time - current_trade['entry_time']).total_seconds() / 3600
    if duration_hours >= MAX_TRADE_HOURS:
        return "TIME_LIMIT", total_pnl, s1_pnl, s2_pnl
    
    return None


In [53]:
def run_backtest(symbol1, symbol2):
    """
    Main function to run the backtest for a given pair of symbols.
    Now uses USD-based profit targets with simple position sizing.
    """
    print(f"\n----- Starting Backtest for {symbol1} / {symbol2} -----")
    
    # 1. Fetch Data
    df1 = get_historical_data(symbol1, TIMEFRAME, START_DATE, END_DATE)
    df2 = get_historical_data(symbol2, TIMEFRAME, START_DATE, END_DATE)

    if df1.empty or df2.empty:
        print(f"Could not fetch data for one of the symbols. Skipping pair.")
        return

    # 2. Combine data and calculate indicators
    df = pd.DataFrame(index=df1.index.union(df2.index))
    df['s1_close'] = df1['close']
    df['s2_close'] = df2['close']
    df.dropna(inplace=True) # Ensure we only have timestamps where both pairs have data

    df['s1_rsi'] = calculate_rsi(df['s1_close'], RSI_PERIOD)
    df['s2_rsi'] = calculate_rsi(df['s2_close'], RSI_PERIOD)
    df['s1_atr'] = calculate_atr(df1['high'], df1['low'], df1['close'], ATR_PERIOD)
    df['s2_atr'] = calculate_atr(df2['high'], df2['low'], df2['close'], ATR_PERIOD)
    df.dropna(inplace=True)

    print(f"Data prepared. Starting simulation with {len(df)} bars.")

    # 3. Simulation Loop
    trade_history = []
    trade_id_counter = 1
    in_trade = False
    current_trade = {}

    for i in range(len(df)):
        # --- CHECK EXIT CONDITIONS ---
        if in_trade:
            # Use the new USD-based exit condition check
            exit_result = check_exit_conditions(
                current_trade, 
                symbol1, 
                symbol2, 
                df.index[i], 
                df['s1_close'].iloc[i], 
                df['s2_close'].iloc[i]
            )
            
            if exit_result is not None:
                exit_reason, total_pnl, s1_pnl, s2_pnl = exit_result
                
                # Close the trade
                current_trade['exit_time'] = df.index[i]
                current_trade['s1_exit_price'] = df['s1_close'].iloc[i]
                current_trade['s2_exit_price'] = df['s2_close'].iloc[i]
                current_trade['exit_reason'] = exit_reason

                duration = (current_trade['exit_time'] - current_trade['entry_time']).total_seconds() / 3600

                # Calculate additional metrics for enhanced reporting
                s1_pips = calculate_pips(symbol1, current_trade['s1_entry_price'], current_trade['s1_exit_price'], current_trade['type'])
                s2_pips = calculate_pips(symbol2, current_trade['s2_entry_price'], current_trade['s2_exit_price'], current_trade['type'])
                
                trade_history.append({
                    # Basic Trade Info
                    "ID": current_trade['id'],
                    "Trade Type": current_trade['type'].upper(),
                    "Entry Time": current_trade['entry_time'],
                    "Exit Time": current_trade['exit_time'],
                    "Duration (hrs)": round(duration, 2),
                    "Exit Reason": exit_reason,
                    
                    # Symbol Information
                    "Symbol1": symbol1,
                    "Symbol2": symbol2,
                    
                    # Entry Conditions (stored during trade opening)
                    "Symbol1 Entry RSI": current_trade.get('s1_entry_rsi', 'N/A'),
                    "Symbol2 Entry RSI": current_trade.get('s2_entry_rsi', 'N/A'),
                    "Symbol1 Entry ATR": current_trade.get('s1_entry_atr', 'N/A'),
                    "Symbol2 Entry ATR": current_trade.get('s2_entry_atr', 'N/A'),
                    
                    # Price Information  
                    "Symbol1 Entry": current_trade['s1_entry_price'],
                    "Symbol1 Exit": current_trade['s1_exit_price'],
                    "Symbol2 Entry": current_trade['s2_entry_price'],
                    "Symbol2 Exit": current_trade['s2_exit_price'],
                    
                    # Position Sizing
                    "Symbol1 Lots": current_trade['s1_lots'],
                    "Symbol2 Lots": current_trade['s2_lots'],
                    "Hedge Ratio": current_trade.get('hedge_ratio', 'N/A'),
                    
                    # Performance Metrics
                    "Symbol1 Pips": round(s1_pips, 1),
                    "Symbol2 Pips": round(s2_pips, 1),
                    "Total Pips": round(s1_pips + s2_pips, 1),
                    "Symbol1 P&L": round(s1_pnl, 2),
                    "Symbol2 P&L": round(s2_pnl, 2),
                    "Total P&L": round(total_pnl, 2)
                })
                
                print(f"  Trade {current_trade['id']} closed: {exit_reason} | Total P&L: ${total_pnl:.2f} | Duration: {duration:.1f}h")
                
                in_trade = False
                current_trade = {}
                continue # Move to next bar after closing

        # --- CHECK ENTRY CONDITIONS ---
        if not in_trade:
            trade_type_to_open = None
            
            # Signal A: Short both pairs
            if df['s1_rsi'].iloc[i] > RSI_OVERBOUGHT and df['s2_rsi'].iloc[i] > RSI_OVERBOUGHT:
                trade_type_to_open = 'short'
            
            # Signal B: Long both pairs
            elif df['s1_rsi'].iloc[i] < RSI_OVERSOLD and df['s2_rsi'].iloc[i] < RSI_OVERSOLD:
                trade_type_to_open = 'long'

            if trade_type_to_open:
                # Calculate ATR values
                s1_atr_val = df['s1_atr'].iloc[i]
                s2_atr_val = df['s2_atr'].iloc[i]
                
                if s1_atr_val > 0 and s2_atr_val > 0:
                    # Use simple position sizing
                    s1_lot_size, s2_lot_size = calculate_simple_lots(
                        symbol1, symbol2, s1_atr_val, s2_atr_val
                    )
                    
                    # Calculate additional metrics for storage
                    s1_pip_size = get_pip_size(symbol1)
                    s2_pip_size = get_pip_size(symbol2)
                    s1_atr_pips = s1_atr_val / s1_pip_size
                    s2_atr_pips = s2_atr_val / s2_pip_size
                    
                    # Calculate hedge ratio
                    hedge_ratio = calculate_hedge_ratio(symbol1, symbol2, s1_atr_val, s2_atr_val)
                    
                    print(f"Entry Signal: {trade_type_to_open}")
                    print(f"  {symbol1}: RSI={df['s1_rsi'].iloc[i]:.1f} | ATR={s1_atr_val:.5f} ({s1_atr_pips:.1f} pips) | Lot size: {s1_lot_size}")
                    print(f"  {symbol2}: RSI={df['s2_rsi'].iloc[i]:.1f} | ATR={s2_atr_val:.5f} ({s2_atr_pips:.1f} pips) | Lot size: {s2_lot_size}")
                    print(f"  Target: ${PROFIT_TARGET_USD} | Stop: ${STOP_LOSS_USD} | Max time: {MAX_TRADE_HOURS}h")
                    
                    # Open the trade with comprehensive information storage
                    in_trade = True
                    current_trade = {
                        # Basic trade info
                        'id': trade_id_counter,
                        'type': trade_type_to_open,
                        'entry_time': df.index[i],
                        
                        # Price and position info
                        's1_entry_price': df['s1_close'].iloc[i],
                        's2_entry_price': df['s2_close'].iloc[i],
                        's1_lots': s1_lot_size,
                        's2_lots': s2_lot_size,
                        
                        # Entry conditions for analysis
                        's1_entry_rsi': round(df['s1_rsi'].iloc[i], 1),
                        's2_entry_rsi': round(df['s2_rsi'].iloc[i], 1),
                        's1_entry_atr': round(s1_atr_val, 5),
                        's2_entry_atr': round(s2_atr_val, 5),
                        
                        # Position sizing info
                        'hedge_ratio': round(hedge_ratio, 4)
                    }
                    trade_id_counter += 1

    # 4. Save Results
    if not trade_history:
        print("No trades were executed for this pair.")
    else:
        report_df = pd.DataFrame(trade_history)
        output_filename = f"{symbol1}_{symbol2}_backtest_report.csv"
        report_df.to_csv(output_filename, index=False)
        print(f"Backtest complete. Report saved to: {output_filename}")
        print("--- Summary ---")
        print(f"Total Trades: {len(report_df)}")
        print(f"Total P&L: ${report_df['Total P&L'].sum():.2f}")
        
        # Enhanced statistics and analysis
        profit_trades = report_df[report_df['Total P&L'] > 0]
        loss_trades = report_df[report_df['Total P&L'] < 0]
        
        print(f"\n--- Detailed Performance Analysis ---")
        print(f"Winning Trades: {len(profit_trades)} ({len(profit_trades)/len(report_df)*100:.1f}%)")
        print(f"Losing Trades: {len(loss_trades)} ({len(loss_trades)/len(report_df)*100:.1f}%)")
        print(f"Average Duration: {report_df['Duration (hrs)'].mean():.1f} hours")
        
        # P&L Statistics
        if len(profit_trades) > 0:
            print(f"Average Win: ${profit_trades['Total P&L'].mean():.2f}")
            print(f"Largest Win: ${profit_trades['Total P&L'].max():.2f}")
        if len(loss_trades) > 0:
            print(f"Average Loss: ${loss_trades['Total P&L'].mean():.2f}")
            print(f"Largest Loss: ${loss_trades['Total P&L'].min():.2f}")
        
        # Exit reason breakdown
        exit_reasons = report_df['Exit Reason'].value_counts()
        print(f"\nExit Reasons: {dict(exit_reasons)}")
        

        
        # Entry Condition Analysis
        long_trades = report_df[report_df['Trade Type'] == 'LONG']
        short_trades = report_df[report_df['Trade Type'] == 'SHORT']
        
        print(f"\nEntry Signal Analysis:")
        print(f"Long Trades: {len(long_trades)} | Short Trades: {len(short_trades)}")
        
        if len(long_trades) > 0:
            try:
                s1_rsi_values = [float(x) for x in long_trades['Symbol1 Entry RSI'] if str(x) != 'N/A']
                s2_rsi_values = [float(x) for x in long_trades['Symbol2 Entry RSI'] if str(x) != 'N/A']
                if s1_rsi_values and s2_rsi_values:
                    avg_rsi1_long = sum(s1_rsi_values) / len(s1_rsi_values)
                    avg_rsi2_long = sum(s2_rsi_values) / len(s2_rsi_values)
                    print(f"Long Trades - Avg Entry RSI: {symbol1}={avg_rsi1_long:.1f}, {symbol2}={avg_rsi2_long:.1f}")
            except (ValueError, TypeError):
                print(f"Long Trades - RSI data not available")
            
        if len(short_trades) > 0:
            try:
                s1_rsi_values = [float(x) for x in short_trades['Symbol1 Entry RSI'] if str(x) != 'N/A']
                s2_rsi_values = [float(x) for x in short_trades['Symbol2 Entry RSI'] if str(x) != 'N/A']
                if s1_rsi_values and s2_rsi_values:
                    avg_rsi1_short = sum(s1_rsi_values) / len(s1_rsi_values)
                    avg_rsi2_short = sum(s2_rsi_values) / len(s2_rsi_values)
                    print(f"Short Trades - Avg Entry RSI: {symbol1}={avg_rsi1_short:.1f}, {symbol2}={avg_rsi2_short:.1f}")
            except (ValueError, TypeError):
                print(f"Short Trades - RSI data not available")
        
        # Position Sizing Analysis
        try:
            avg_s1_lots = sum(report_df['Symbol1 Lots']) / len(report_df['Symbol1 Lots'])
            avg_s2_lots = sum(report_df['Symbol2 Lots']) / len(report_df['Symbol2 Lots'])
            hedge_ratios = [float(x) for x in report_df['Hedge Ratio'] if str(x) != 'N/A']
            avg_hedge_ratio = sum(hedge_ratios) / len(hedge_ratios) if hedge_ratios else 0
            
            print(f"\nPosition Sizing Analysis:")
            print(f"Average Lot Sizes: {symbol1}={avg_s1_lots:.2f}, {symbol2}={avg_s2_lots:.2f}")
            print(f"Average Hedge Ratio: {avg_hedge_ratio:.3f}")
        except (ValueError, TypeError, ZeroDivisionError):
            print(f"\nPosition Sizing Analysis: Data not available")
        
        # Pips vs USD Analysis
        total_pips = report_df['Total Pips'].sum()
        total_usd = report_df['Total P&L'].sum()
        print(f"\nPips vs USD Comparison:")
        print(f"Total Pips: {total_pips:.1f} | Total USD: ${total_usd:.2f}")
        print(f"Average USD per Pip: ${total_usd/total_pips:.2f}" if total_pips != 0 else "No pip data available")


In [54]:
# Duplicate function removed - using the updated USD-based version in cell above


In [55]:
# Second duplicate function removed - using the updated USD-based version above


In [56]:
# --- Running Backtest on All Pairs ---
print("=== Starting Backtest for All 69 Pairs ===\n")
print("This will take significant time (potentially several hours)...")
print("Please ensure MT5 stays connected and your system doesn't sleep.\n")

# Track progress
total_pairs = len(PAIRS_TO_TEST)
completed_pairs = 0
skipped_pairs = 0

for pair_info in PAIRS_TO_TEST:
    s1, s2, _ = pair_info
    
    # Ensure symbols are available in MT5
    s1_info = mt5.symbol_info(s1)
    s2_info = mt5.symbol_info(s2)
    
    if s1_info is None:
        print(f"Symbol {s1} not found in MT5, skipping pair.")
        skipped_pairs += 1
        continue
    if s2_info is None:
        print(f"Symbol {s2} not found in MT5, skipping pair.")
        skipped_pairs += 1
        continue
        
    run_backtest(s1, s2)
    completed_pairs += 1
    print(f"Progress: {completed_pairs}/{total_pairs - skipped_pairs} pairs completed")

print(f"\n\n----- All Backtests Completed -----")
print(f"Successfully processed: {completed_pairs} pairs")
print(f"Skipped pairs: {skipped_pairs}")
print(f"Total CSV reports generated: {completed_pairs}")


=== Starting Backtest for All 69 Pairs ===

This will take significant time (potentially several hours)...
Please ensure MT5 stays connected and your system doesn't sleep.


----- Starting Backtest for USDJPY / AUDUSD -----
Data prepared. Starting simulation with 70071 bars.
Entry Signal: short
  USDJPY: RSI=79.2 | ATR=0.68920 (68.9 pips) | Lot size: 1.0
  AUDUSD: RSI=79.9 | ATR=0.00031 (3.1 pips) | Lot size: 5.0
  Target: $500.0 | Stop: $-15000.0 | Max time: 2400h
  Trade 1 closed: PROFIT_TARGET | Total P&L: $3719.09 | Duration: 216.0h
Entry Signal: short
  USDJPY: RSI=81.4 | ATR=0.58200 (58.2 pips) | Lot size: 1.0
  AUDUSD: RSI=93.2 | ATR=0.00023 (2.3 pips) | Lot size: 5.0
  Target: $500.0 | Stop: $-15000.0 | Max time: 2400h
  Trade 2 closed: PROFIT_TARGET | Total P&L: $867.08 | Duration: 48.0h
Entry Signal: short
  USDJPY: RSI=76.6 | ATR=0.65300 (65.3 pips) | Lot size: 1.0
  AUDUSD: RSI=79.4 | ATR=0.00026 (2.6 pips) | Lot size: 5.0
  Target: $500.0 | Stop: $-15000.0 | Max time: 2400

In [57]:
# --- Shutdown MT5 Connection ---
mt5.shutdown()
print("MetaTrader5 connection shut down.")


MetaTrader5 connection shut down.
